# 06. RAG_DocumentGPT

## RAG (Retrieval Augmented Generation, 검색 증강 생성)

- model들은 많은 데이터를 통해 학습하지만 DB나 문서와 같은 private data에 접근할 수는 없다.
- model은 이러한 data를 알 수 없으므로, 우리는 질문할 때 그와 관련있는 문서들도 같이 준비하게 된다. (vector나 저장기 활용은 뒤에서 배운다.)

### What is 'foo'?

- 우리의 model은 'foo'라는 단어를 아예 모르는 상태이다.
- 따라서 우리는 'foo'와 관련된 문서들을 준비해 가져올 것이다.
- **이 문서들을 context로써 질문과 함께 묶어 prompt를 구성해, Language Model에 보낼 것이다.**
- Model은 기존에 학습된 data와 추가로 전달 받은 context로써의 data를 모두 갖게 된다.
- 이러한 개념을 ***RAG*** 이라고 한다.

## Retrieval
- Langchain의 모듈
- Source -> Load -> Transform -> Embed -> Store -> Retrieve
  
### UnstructedFileLoader
- 대부분 파일들에 호환되는 범용 loader

### 문서 분할

- 예제의 경우 전체 챕터가 하나의 문서에 들어가 있다. 
- 너무 큰 한 덩어리 데이터를 분할하여 파일의 필요한 부분만을 사용하고자 분할한다.
- `chunk_size`를 지정해 잘라낸 각 덩어리는 작아졌지만 문단까지 잘려 그냥 사용하기엔 문제가 많았음.
- `chunk_overlap` 을 지정해 앞 chunk의 일부를 뒤 chunk의 시작으로 끌고올 수 있다. 단, 이로 인해 chunk 간 중복되는 부분이 생긴다.
- 

### Token

위에서 text를 chunk 단위로 분류했지만, 엄밀히 token과 letter는 다른 것이다.
- token은 여러 개의 문자나 단어가 될 수도 있다.

문장을 Token으로 분리했을 때, Model이 받게 되는 데이터는 각 Token에 대한 ID값이다.
- nomad : nom / ad -> [17101, 329] 와 같은 식
- 이를 기준으로 AI는 'token_17101 다음에 token_329가 올 확률이 높다' 등의 예측 가능

### Tiktoken

이러한 데이터의 Tokenizing을 위해 사용하는 라이브러리

## Embedding

지금까지 위의 RAG 중 Load-Transform 단계까지는 수행했다.   
이제 컴퓨터가 이해할 수 있도록 data를 Embedding해 주어야 한다.

문서의 split 처리된 data마다 각각의 (3D)vector를 만들 것이며, 이를 위해 embedding model을 사용한다.

Masculinity | Femininity | Royalty 를 우리의 3개 차원으로 둔다고 가정해 보자.
여러 단어들을 아래와 같이 우리의 차원에서 정의해 보자.

[word(Masculinity, Femininity, Royalty)]
- King  (0.9, 0.1, 1.0)
- Queen (0.1, 0.9, 1.0)
- Man   (0.9, 0.1, 0,0)

'우리의 차원에서 King은 0.9의 남성성, 1.0의 충성심, 0.1의 여성성을 가진다' 라는 것을 3차원 vector로 나타내었다.   

- Vector Calculus
King-Man = (0, 0, 1.0) = ***Royal***   
이는 'King과 Man은 우리의 차원에서 Royalty=1.0의 차이를 갖는다.' 로 해석할 수 있으며,   
우리의 차원에서 이러한 단어는 'royal'과 같이 정의내릴 수 있다.   

Man+Royal = (0.9, 0.1, 1.0) = ***King***   
이처럼 단어 간의 연관성(가깝고 멂)을 벡터 연산을 통해 나타낼 수 있다.   
(+) : 벡터 간 연관성은 벡터 간 거리로 판단한다.   

이를 통해 특정 단어에 대해 연관성이 높은 단어를 **Search**할 수 있게 된다.   
(+) 물론 연관성의 range에 대한 것은 개발자가 정해야 할 것 같다.   

많은 추천 알고리즘은 이와 같은 방식으로 동작한다.   


차원의 개수가 늘어날수록 한 단어가 담고 있는(표현할 수 있는) 속성의 개수가 많을 것이고,   
이를 통해 비슷한 영역에 있는 단어나 문서를 찾을 수 있다.   
(참고 : The conspiracy to make AI seem harder than it is! By Gustav Soderstrom (Spotify R&D) (Youtube))


## Vector Store

일종의 DB로 생각할 수 있다. Embed를 통해 vector를 만들고 이곳에 저장한 후,   
vector store에서 검색할 수 있게 된다. (관련있는 문서만 찾아낼 수 있게 된다.)

매 코드 실행마다 문서를 embed하기 보다는, embed한 내용을 저장하여 사용하자.   
Langchain에서는 embed 결과를 cache에 저장할 수 있도록 기능을 제공해 준다.

우리는 Chroma를 사용할 것이다. Local에서 실행될 것임.   

vectorstore로 보내지는 문서의 크기에 따라 비용이 달라지므로,   
의미가 왜곡되지 않는 선에서 적정한 split chunksize를 정하는 것이 중요할 것으로 보임.


In [1]:
##import os

##tiktoken_cache_dir = "./tiktoken_cache"
##os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir


from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory

from langchain.callbacks import StreamingStdOutCallbackHandler

## import tiktoken_ext.openai_public


## Document 분할
##from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.vectorstores import Chroma

from langchain.storage import LocalFileStore

import tiktoken

## loader import
from langchain.document_loaders import UnstructuredFileLoader ##TextLoader, PyPDFLoader 등 범용

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,     ##Streaming 옵션 ON
    callbacks=[StreamingStdOutCallbackHandler()]
)




In [2]:
import tiktoken_ext.openai_public
import inspect

print(dir(tiktoken_ext.openai_public))
# The encoder we want is cl100k_base, we see this as a possible function

print(inspect.getsource(tiktoken_ext.openai_public.cl100k_base))
# The URL should be in the 'load_tiktoken_bpe function call'

['ENCODING_CONSTRUCTORS', 'ENDOFPROMPT', 'ENDOFTEXT', 'FIM_MIDDLE', 'FIM_PREFIX', 'FIM_SUFFIX', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cl100k_base', 'data_gym_to_mergeable_bpe_ranks', 'gpt2', 'load_tiktoken_bpe', 'o200k_base', 'p50k_base', 'p50k_edit', 'r50k_base']
def cl100k_base():
    mergeable_ranks = load_tiktoken_bpe(
        "https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken",
        expected_hash="223921b76ee99bde995b7ff738513eef100fb51d18c93597a113bcffe865b2a7",
    )
    special_tokens = {
        ENDOFTEXT: 100257,
        FIM_PREFIX: 100258,
        FIM_MIDDLE: 100259,
        FIM_SUFFIX: 100260,
        ENDOFPROMPT: 100276,
    }
    return {
        "name": "cl100k_base",
        "pat_str": r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""",
        "mergeable_ranks": mergeable_ranks,
        "special_tokens": speci

In [5]:
encoding = tiktoken.get_encoding("o200k_base")
encoding.encode("Hello")

[13225]

In [5]:
import certifi
import urllib3

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
response = http.request('GET', 'https://openaipublic.blob.core.windows.net/gpt-2/encodings/main/vocab.bpe')
print(response.data)

b'#version: 0.2\n\xc4\xa0 t\n\xc4\xa0 a\nh e\ni n\nr e\no n\n\xc4\xa0t he\ne r\n\xc4\xa0 s\na t\n\xc4\xa0 w\n\xc4\xa0 o\ne n\n\xc4\xa0 c\ni t\ni s\na n\no r\ne s\n\xc4\xa0 b\ne d\n\xc4\xa0 f\nin g\n\xc4\xa0 p\no u\n\xc4\xa0a n\na l\na r\n\xc4\xa0t o\n\xc4\xa0 m\n\xc4\xa0o f\n\xc4\xa0 in\n\xc4\xa0 d\n\xc4\xa0 h\n\xc4\xa0an d\ni c\na s\nl e\n\xc4\xa0t h\ni on\no m\nl l\nen t\n\xc4\xa0 n\n\xc4\xa0 l\ns t\n\xc4\xa0 re\nv e\n\xc4\xa0 e\nr o\nl y\n\xc4\xa0b e\n\xc4\xa0 g\n\xc4\xa0 T\nc t\n\xc4\xa0 S\ni d\no t\n\xc4\xa0 I\nu t\ne t\n\xc4\xa0 A\n\xc4\xa0 is\n\xc4\xa0 on\ni m\na m\no w\na y\na d\ns e\n\xc4\xa0th at\n\xc4\xa0 C\ni g\n\xc4\xa0f or\na c\n\xc4\xa0 y\nv er\nu r\n\xc4\xa0 u\nl d\n\xc4\xa0s t\n\xc4\xa0 M\n\' s\n\xc4\xa0 he\n\xc4\xa0 it\nat ion\nit h\ni r\nc e\n\xc4\xa0y ou\ni l\n\xc4\xa0 B\n\xc4\xa0w h\no l\n\xc4\xa0 P\n\xc4\xa0w ith\n\xc4\xa0 1\nt er\nc h\n\xc4\xa0a s\n\xc4\xa0w e\n\xc4\xa0 (\nn d\ni ll\n\xc4\xa0 D\ni f\n\xc4\xa0 2\na g\ner s\nk e\n\xc4\xa0 "\n\xc4\xa0 H\ne m\n\xc4\x

In [6]:

splitterTiktoken = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",     ## CharacterTextSplitter에 한해 사용
    chunk_size=600,     ## split size
    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴   
)

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /gpt-2/encodings/main/vocab.bpe (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

### Issue

CharacterTextSplitter.from_tiktoken_encoder 사용 시 SSL Error 발생   
사내망이 아닌 외부망에서 테스트 예정 (아래 예제는 일반 splitter로 실행하였음)   

In [4]:
loader = UnstructuredFileLoader("./files/document.txt")

# validate
##assert os.path.exists(os.path.join(tiktoken_cache_dir,"9b5ad71b2ce5302211f9c61530b329a4922fc6a4"))

b'#version: 0.2\n\xc4\xa0 t\n\xc4\xa0 a\nh e\ni n\nr e\no n\n\xc4\xa0t he\ne r\n\xc4\xa0 s\na t\n\xc4\xa0 w\n\xc4\xa0 o\ne n\n\xc4\xa0 c\ni t\ni s\na n\no r\ne s\n\xc4\xa0 b\ne d\n\xc4\xa0 f\nin g\n\xc4\xa0 p\no u\n\xc4\xa0a n\na l\na r\n\xc4\xa0t o\n\xc4\xa0 m\n\xc4\xa0o f\n\xc4\xa0 in\n\xc4\xa0 d\n\xc4\xa0 h\n\xc4\xa0an d\ni c\na s\nl e\n\xc4\xa0t h\ni on\no m\nl l\nen t\n\xc4\xa0 n\n\xc4\xa0 l\ns t\n\xc4\xa0 re\nv e\n\xc4\xa0 e\nr o\nl y\n\xc4\xa0b e\n\xc4\xa0 g\n\xc4\xa0 T\nc t\n\xc4\xa0 S\ni d\no t\n\xc4\xa0 I\nu t\ne t\n\xc4\xa0 A\n\xc4\xa0 is\n\xc4\xa0 on\ni m\na m\no w\na y\na d\ns e\n\xc4\xa0th at\n\xc4\xa0 C\ni g\n\xc4\xa0f or\na c\n\xc4\xa0 y\nv er\nu r\n\xc4\xa0 u\nl d\n\xc4\xa0s t\n\xc4\xa0 M\n\' s\n\xc4\xa0 he\n\xc4\xa0 it\nat ion\nit h\ni r\nc e\n\xc4\xa0y ou\ni l\n\xc4\xa0 B\n\xc4\xa0w h\no l\n\xc4\xa0 P\n\xc4\xa0w ith\n\xc4\xa0 1\nt er\nc h\n\xc4\xa0a s\n\xc4\xa0w e\n\xc4\xa0 (\nn d\ni ll\n\xc4\xa0 D\ni f\n\xc4\xa0 2\na g\ner s\nk e\n\xc4\xa0 "\n\xc4\xa0 H\ne m\n\xc4\x

In [5]:


## embedded data의 cache 저장을 위한 storage 생성 (local)
cache_dir = LocalFileStore("./.cache/")

## 파일 분할 Splitter (문장 끝이나 문단 끝부분마다 잘라줌)
"""
splitter = CharacterTextSplitter(
    separator="",     ## CharacterTextSplitter에 한해 사용
    chunk_size=600,     ## split size
    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴
    ##참고 : len은 기본 라이브러리 함수임
)
"""


'\nsplitter = CharacterTextSplitter(\n    separator="",     ## CharacterTextSplitter에 한해 사용\n    chunk_size=600,     ## split size\n    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴\n    ##참고 : len은 기본 라이브러리 함수임\n)\n'

In [6]:
embedder = OpenAIEmbeddings()

##vector = embedder.embed_query("Hi")
## 아래의 output은 단어 'Hi'의 vector값이다.

## 총 1536개의 차원으로 단어를 표현한다는 의미이다.
## len(vector)

docs = loader.load_and_split(text_splitter=splitterTiktoken)





Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [9]:
## vectorstore에서 유사도 검색을 수행 (similarity_search)
## 우리 문서의 단어를 vector로 변환해 공간을 구성하고, 이 공간에서 단어 간의 연관성을 찾음
vectorstore = Chroma.from_documents(docs, embedder)

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [ ]:
## Cache Embedding (CacheBackedEmbedding는 Embedder를 받아야 함
## chche hit -> load from cache_dir, cache miss -> embed on embeddings
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder, cache_dir
)

## Cache Embedding은 아래와 같이 Chroma를 통해 실행한다.
## vectorstore = Chroma.from_documents(docs, cached_embeddings)



## Langsmith

(https://smith.langchain.com/)

## RetrievalQA

- 앞선 강의에서 사용했던 LLMChain은 Legacy에 해당함
- 따라서 Langchain에서는 LCLE Chain을 사용하는 것을 권장하고 있음.

- off-the-shelf chain으로 property를 전달하면 Document chain이 만들어져 즉시 답변됨
- 하지만 다소 모호하고, 커스터마이징이 어려움.

- System Message는 자동으로 넣어주는 것으로 확인됨.

### Document chain의 생성 방법

1. Stuff
- document로 prompt를 채우는(stuffing) 단순한 방법
- 사용자의 질문에 문서를 별도 프로세싱 없이 엎어 넘기는 방법

2. Refine
- 질문과 관련된 document를 얻어, **각각의 document를 읽으면서 질문에 대한 답변 생성을 시도**함.
- 만나는 모든 document를 통해 question을 개선시킨다.
- 즉, 질문만을 기반으로 첫 답변을 생성하고, retrieve된 document를 차례로 보며 답변을 refine해가는 방식이다.
- chain 내부에서 document의 수만큼 질의를 수행하므로 cost가 더 높음.

3. Map Reduce
- 질문에 대한 Document들을 입력받아 개별적으로 요약 작업을 수행한다.
- 각각의 요약본을 LLM에 전달한다.

4. Map Re-Rank
- 질문에 대한 Document들을 입력받아 각각의 답변을 생성하고, **각 답변에 점수를 부여**한다.
- 최종적으로 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환한다.

(10:45) Chroma를 FAISS로 바꿔주었음

In [8]:
from langchain.chains import RetrievalQA

## chain의 생성자이며, llm을 인자로 받음
"""
    A retriever is an interface that returns documents given an unstructured query. 
    It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. 
    Retrievers can be created from vector stores, but are also broad enough to include Wikipedia search and Amazon Kendra.
"""
chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff", ## str을 받으며 default값은 stuff임,
    retriever=vectorstore.as_retriever(),      ## retriever는 class의 interface와 같다. (vectorstore를 포함한, document가 저장된 db나 cloud 환경을 일컫는다.)
)

chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")


NameError: name 'vectorstore' is not defined